# ¿Cómo se comportan los sacks permitidos por los equipos (y sus QB/línea ofensiva) en los partidos decididos por 8 puntos o más?

In [1]:
import pandas as pd

In [3]:

df = pd.read_csv("raw_data/merged_games_boxscores.csv")
df.rename(columns={"season_x": "season",
                    "Unnamed: 5": "Location",
                    "Los...D.2": "Loser/tie"},
                    inplace=True)
if "season_y" in df.columns:
    df.drop(columns=["season_y"], inplace=True)

df["season"] = pd.to_numeric(df["season"], errors="coerce").astype("Int64")
df["margin"] = pd.to_numeric(df["margin"], errors="coerce")
df["fecha"]  = pd.to_datetime(df["file"].str.extract(r"(\d{8})")[0],format="%Y%m%d", errors="coerce")

# 2. Asegurarte de tener siempre 'versus'
#    (lo creaste así en merged_df)
# df["versus"] = df["local"] + " vs " + df["visitante"]

records = []
# 3. Agrupa por fecha y por versus
for (fecha, versus), group in df.groupby(["fecha", "versus"]):
    game   = group.iloc[0]
    margin = abs(game["margin"])
    if margin < 8:
        continue

    local_name, visitante_name = versus.split(" vs ")
    equipos_abbr = group["Tm"].dropna().unique().tolist()
    local_abbr, visitor_abbr = equipos_abbr

    for equipo in equipos_abbr:
        team_group = group[group["Tm"] == equipo]
        is_home    = (equipo == local_abbr)
        resultado  = "win" if (
            (is_home and local_name  in game["Winner/tie"]) or
            (not is_home and visitante_name in game["Winner/tie"])
        ) else "loss"

        puntos_anotados = (
            game["PtsWinner"] if resultado == "win"
            else game["PtsLosser"]
        )
        sacks_permitidos = team_group["Sk"].fillna(0).sum()
        jugadores = (team_group.loc[team_group["Sk"].fillna(0)>0, "Player"].dropna().unique().tolist())
        att = team_group["Att"].fillna(0).sum()
        # sacks_qb = team_group.loc[(team_group["Cmp"].fillna(0)>0) & (team_group["Sk"].fillna(0)>0),"Sk"].sum()
        pct_sacks = sacks_permitidos / att if att > 0 else None
        # sk_qb_ratio = sacks_qb / sacks_permitidos if sacks_permitidos > 0 else None
        sk_margin_ratio = sacks_permitidos / margin if margin > 0 else None

        records.append({
            "season": game["season"],
            "Week": game["Week"],
            "phase": game["phase"],
            "fecha": fecha,
            "partido_id": versus,
            "team_abbr": equipo,
            "local_team_name": local_name,
            "visitante_team_name": visitante_name,
            "is_home":   is_home,
            "resultado": resultado,
            "margin":    margin,
            "puntos_anotados": puntos_anotados,
            "sacks_permitidos": sacks_permitidos,
            "jugadores_implicados": len(jugadores),
            "nombres_jugadores": jugadores,
            "att": att,
            # "sacks_qb": sacks_qb,
            "pct_sacks": pct_sacks,
            # "sk_qb_ratio": sk_qb_ratio,
            "sk_margin_ratio": sk_margin_ratio,
        })

# 4. DataFrame final
df_processed = pd.DataFrame(records)
df_processed.to_csv("processed_data.csv", index=False)
df_processed

,season,Week,phase,fecha,partido_id,team_abbr,local_team_name,visitante_team_name,is_home,resultado,margin,puntos_anotados,sacks_permitidos,jugadores_implicados,nombres_jugadores,att,pct_sacks,sk_margin_ratio
0,2020,1,regular,2020-09-10,Kansas City Chiefs vs Houston Texans,HOU,Kansas City Chiefs,Houston Texans,True,win,14,34,4.0,1,[Deshaun Watson],32.0,0.125000,0.285714
1,2020,1,regular,2020-09-10,Kansas City Chiefs vs Houston Texans,KAN,Kansas City Chiefs,Houston Texans,False,loss,14,20,1.0,1,[Patrick Mahomes],32.0,0.031250,0.071429
2,2020,1,regular,2020-09-13,Atlanta Falcons vs Seattle Seahawks,SEA,Atlanta Falcons,Seattle Seahawks,True,loss,13,25,3.0,1,[Russell Wilson],35.0,0.085714,0.230769
3,2020,1,regular,2020-09-13,Atlanta Falcons vs Seattle Seahawks,ATL,Atlanta Falcons,Seattle Seahawks,False,win,13,38,2.0,1,[Matt Ryan],54.0,0.037037,0.153846
4,2020,1,regular,2020-09-13,Baltimore Ravens vs Cleveland Browns,CLE,Baltimore Ravens,Cleveland Browns,True,win,32,38,2.0,1,[Baker Mayfield],39.0,0.051282,0.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1439,2024,Division,playoff,2025-01-18,Kansas City Chiefs vs Houston Texans,KAN,Kansas City Chiefs,Houston Texans,False,loss,9,14,3.0,1,[Patrick Mahomes],25.0,0.120000,0.333333
1440,2024,ConfChamp,playoff,2025-01-26,Philadelphia Eagles vs Washington Commanders,WAS,Philadelphia Eagles,Washington Commanders,True,win,32,55,3.0,1,[Jayden Daniels],49.0,0.061224,0.093750
1441,2024,ConfChamp,playoff,2025-01-26,Philadelphia Eagles vs Washington Commanders,PHI,Philadelphia Eagles,Washington Commanders,False,loss,32,23,2.0,1,[Jalen Hurts],28.0,0.071429,0.062500
1442,2024,SuperBowl,playoff,2025-02-09,Philadelphia Eagles vs Kansas City Chiefs,KAN,Philadelphia Eagles,Kansas City Chiefs,True,win,18,40,6.0,1,[Patrick Mahomes],32.0,0.187500,0.333333


In [2]:
df = pd.read_csv("processed_data.csv", parse_dates=["fecha"])
df8 = df[df["margin"].abs() >= 8].copy()

if "pct_sacks" not in df8.columns:
    att_column = next((c for c in ["att", "pass_attempts"] if c in df8.columns), None)
    if att_column is None:
        raise KeyError(
            "No encuentro columna de intentos de pase ('att' o 'pass_attempts') "
            "para calcular pct_sacks."
        )
    df8["dropbacks"] = df8[att_column] + df8["sacks_permitidos"]
    df8["pct_sacks"] = df8["sacks_permitidos"] / df8["dropbacks"]


sacks_mean = df8.groupby("resultado")["sacks_permitidos"].mean()
pct_mean   = df8.groupby("resultado")["pct_sacks"].mean()
rho        = df8["sacks_permitidos"].corr(df8["margin"])

print(" Promedio de SACKS por resultado ")
print(f"\tGANADORES: {sacks_mean['win'] :.2f}")
print(f"\tPERDEDORES: {sacks_mean['loss']:.2f}\n")

print(" % de DROPBACKS que terminan en SACKS ")
print(f"\tGANADORES: {pct_mean['win']  *100:5.1f} %")
print(f"\tPERDEDORES: {pct_mean['loss']*100:5.1f} %\n")

print("Correlación SACKS ↔ MARGEN (Pearson)")
print(f"  ρ = {rho:.3f}\n")

top_pressure = (
    df8[df8["resultado"] == "win"]
    .groupby("team_abbr")["sacks_permitidos"]
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

print("SACKS medios en VICTORIAS ≥ 8 pts (Top-10)")
display(top_pressure)

summary = pd.Series(
    {
        "avg_sacks_win":  sacks_mean["win"],
        "avg_sacks_loss": sacks_mean["loss"],
        "rho_sacks_margin": rho,
        "pct_sacks_win":  pct_mean["win"],
        "pct_sacks_loss": pct_mean["loss"],
    }
)


 Promedio de SACKS por resultado 
	GANADORES: 3.26
	PERDEDORES: 1.68

 % de DROPBACKS que terminan en SACKS 
	GANADORES:   9.8 %
	PERDEDORES:   6.0 %

Correlación SACKS ↔ MARGEN (Pearson)
  ρ = -0.027

SACKS medios en VICTORIAS ≥ 8 pts (Top-10)


team_abbr
NYG    4.083333
CIN    3.933333
CLE    3.846154
HOU    3.727273
CHI    3.718750
BAL    3.571429
CAR    3.512821
PHI    3.500000
NYJ    3.485714
WAS    3.482759
Name: sacks_permitidos, dtype: float64